### [Get Started with PySpark and Jupyter Notebook in 3 Minutes](https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f)

### Importing Libraries
  * #### Part 1: Essential Libraries to
    * Find Spark Installation &nbsp;&nbsp;&nbsp;&nbsp;>>> **import** findspark
    * Initiate a Spark Instance &nbsp;>>> **findspark.init()**

In [1]:
import findspark; findspark.init()

  * #### Part 2: Importing Essential pyspark Modules

In [2]:
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark import SQLContext
import time

### Loading Spark and SQL Context

In [3]:
sc = SparkContext("local[*]", "db2-q2-spark-df-a")
sqlContext = SQLContext(sc)

### Create Dataset from csv file --  SQLContext

In [8]:
log_records = sqlContext.read.load("db2_project_data.csv", format='com.databricks.spark.csv', header='true', inferSchema='true')
log_records.persist() # Make it persistent to both MEMORY (if it "fits") and DISK to speed up computations

DataFrame[id: string, timestamp: timestamp, kilometers: double, region_id: int]

### Compute Statistics for our Dataset

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1. Number of Distinct Cars 

In [13]:
st = time.time()
num = log_records.select('ID').distinct().count()

print ('Operation took:', time.time() - st, ' seconds to complete.')
print ('The number of distinct cars is: ', num)

Operation took: 0.7978129386901855  seconds to complete.
The number of distinct cars is:  1596


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2. Number of Cars that Passed each Region

In [15]:
st = time.time()
res = log_records.groupby("region_id").agg({'ID': 'count'})
fn = time.time() - st

print ('Query Computation Time:', fn, ' seconds.')
print ('The Number of Cars that Passed each Region:')
print (res.show(3))
print ('Cumulative Computation Time:', time.time() - st, ' seconds.')

Query Computation Time: 0.010528326034545898  seconds.
The Number of Cars that Passed each Region:
+---------+---------+
|region_id|count(ID)|
+---------+---------+
|       31|   108261|
|       53|   108960|
|       34|   108540|
+---------+---------+
only showing top 3 rows

None
Cumulative Computation Time: 0.2912750244140625  seconds.


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3. Traffic Percentage in each Region 

Approach - Using Dataframe Operations
  * Using Native SQLContext Functions we group by the region id and count the number of cars that passed from that single region

In [18]:
st = time.time()
num = log_records.count()
res = log_records.groupby('region_id').agg((count(log_records['ID'])/num*100).alias('percentage'))
fn = time.time() - st

print ('Query Computation Time:', fn, ' seconds.')
print ('The Traffic Percentage in each Region:')
print (res.show(3))
print ('Cumulative Computation Time:', time.time() - st, ' seconds.')

Query Computation Time: 0.11533498764038086  seconds.
The Traffic Percentage in each Region:
+---------+------------------+
|region_id|        percentage|
+---------+------------------+
|       31|1.8480647393665566|
|       53|1.8599969887713947|
|       34|1.8528273968543243|
+---------+------------------+
only showing top 3 rows

None
Cumulative Computation Time: 0.37951016426086426  seconds.


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4. Kilometer Percentage per Month

In [21]:
st = time.time()
num = log_records.agg(sum(log_records['kilometers'])).collect()[0][0]
res = log_records.groupby(month(log_records['timestamp']).alias('month')).agg((sum(log_records['kilometers'])/num*100).alias('percentage'))
fn = time.time() - st

print ('Query Computation Time:', fn, ' seconds.')
print ('The kilometer Distance Percentage per month:')
print (res.show(3))
print ('Cumulative Computation Time:', time.time() - st, ' seconds.')

Query Computation Time: 1.1453475952148438  seconds.
The kilometer Distance Percentage per month:
+-----+------------------+
|month|        percentage|
+-----+------------------+
|   12| 13.74256831097875|
|    1|14.162338451444722|
|    6|13.840445194874022|
+-----+------------------+
only showing top 3 rows

None
Cumulative Computation Time: 1.3829827308654785  seconds.


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5. Average Kilometers per Month

In [24]:
st = time.time()
res = log_records.groupby([log_records['region_id'], month(log_records['timestamp']).alias('month')]).agg(avg(log_records['kilometers']).alias('avg_kilometers'))
fn = time.time() - st

print ('Query Computation Time:', fn, ' seconds.')
print ('The Average Kilometer Distance in each Region per Month:')
print (res.show(3))
print ('Cumulative Computation Time:', time.time() - st, ' seconds.')

Query Computation Time: 0.01503896713256836  seconds.
The Average Kilometer Distance in each Region per Month:
+---------+-----+-----------------+
|region_id|month|   avg_kilometers|
+---------+-----+-----------------+
|       43|    7|550.0570712663371|
|       10|    2|564.7339467337513|
|       41|    3|562.3467249354493|
+---------+-----+-----------------+
only showing top 3 rows

None
Cumulative Computation Time: 0.6021502017974854  seconds.


### Stopping Spark Context... 

In [25]:
sc.stop()